<a href="https://colab.research.google.com/github/rasik-nep/Chatbot-Prototype/blob/main/Nepali_Text_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Application uses OpenAI and Langchain to create a chatbot for text in Nepali language. It includes vector embedding for better understanding and response.

In [ ]:
!pip install langchain OpenAI langchain_openai load_dotenv chromadb

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAI

In [ ]:
documents = TextLoader("citizenship.txt").load()
documents

In [8]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [21]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

# we are specifying that OpenAI is the LLM that we want to use in our chain
chain = load_qa_chain(llm=OpenAI())
query = 'who are allowed to get citizenship in nepal?'
response = chain.invoke({"input_documents": documents, "question": query})
print(response["output_text"])

 According to the context provided, any Nepali citizen is eligible to receive citizenship in Nepal, including those who have permanent residence in the country, those who have Nepali parentage, and those who have been born in Nepal. However, there are also provisions for foreign nationals to obtain Nepali citizenship under certain circumstances, such as through marriage to a Nepali citizen or through recognition as a Nepali citizen by the government.


In [24]:
# we are specifying that OpenAI is the LLM that we want to use in our chain
chain = load_qa_chain(llm=OpenAI())
query = 'नेपालमा नागरिकता कसले कसले पाउने ?'
response = chain.invoke({"input_documents": documents, "question": query})
print(response["output_text"])


नेपालमा नागरिकता पाउने लागि वंशजको आधारमा नेपालको नागरिक हुनु पर्छ । कुनै व्यक्तिको जन्म हुँदाका बखत निजको बाबु वा आमा नेपालको नागरिक रहेछ भने त्यस्तो व्यक्ति र जन्मको आधारमा नेपालको नागरिकता प्राप्त गर्न सक्नुहुन्छ । यो संविधान प्रारम्भ हुनुभन्दा अघि जन


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=len
)
texts = text_splitter.split_documents(documents)
texts

In [27]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [28]:
vectordb = Chroma.from_documents(
  texts,
  embedding=OpenAIEmbeddings(),
  persist_directory='./data'
)
vectordb.persist()

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

# we can now execute queries against our Q&A chain
result = qa_chain.invoke({'query': 'who are allowed to get citizenship in nepal?'})
print(result['result'])

 People who are permanent residents of Nepal based on their ancestry or those who are born in Nepal to a Nepali citizen parent, even if they don't have a Nepali citizenship themselves, are allowed to obtain Nepali citizenship. Additionally, those who are married to a Nepali citizen or have Nepali ancestry also have the option to obtain Nepali citizenship.


In [30]:
result = qa_chain.invoke({'query': 'नेपालमा नागरिकता कसले कसले पाउने ?'})
print(result['result'])

 नेपालमा नागरिकता प्राप्त गर्न सक्ने कुनै कुनै व्यक्तिहरु यहाँ दिएको विषयसँग सम्बन्धित हुन्छन्: जन्मको आधारमा नेपालको नागरिकता प्राप्त गर्न सक्ने व्यक्तिहरू, जस्तैकि यो संविधान प्रारम्भ हुनुभन्दा अघि जन्मको आधारमा नेपालको नागरिकता पाप्त गरेको नागरिकको सन्तानले
